In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os
np.random.seed(0) #Code B

In [2]:
N = 100
A =  np.random.randint(2, size=N)
B =  np.random.randint(2, size=N)
C =  np.random.randint(2, size=N)

customer = ['c'+str(i+16000) for i in range(90)]

Fraud = A & B & C
df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "C":C, "Fraud":Fraud})


In [3]:
df.drop(df[(A==0)&(B==0)&(C==0)].index, inplace=True)
df.drop(df[df['orig']==df['destination']].index, inplace=True)


In [4]:
fraudsters = df[df['Fraud']==True]

In [5]:
fraudsters[['orig','destination']]

,orig,destination
10,c16053,c16055
28,c16071,c16013
35,c16088,c16011
41,c16034,c16052
44,c16080,c16018
46,c16008,c16051
56,c16035,c16070
76,c16071,c16079
90,c16000,c16004


### Erman's Idea

Target is Fraud  (1) iff A and B and C and D (are 1)

Main idea: We need to learn a rule: 

Fraud_trans(U1, U2):- Transaction(X), Orig(X, U1), Dest(X, U2),  Fraud(X).

Fraud_chain(U1, U2):- Fraud_trans(U1, Y), Fraud_chain(Y, U2),



In [6]:
index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), how='inner', left_on='orig', right_on='destination')

In [7]:
index_from_merge

,index,orig,destination_x,A,B,C,Fraud,destination_y
0,7,c16051,c16013,1,1,0,0,c16051
1,98,c16051,c16023,1,0,1,0,c16051
2,60,c16018,c16080,1,0,1,0,c16018
3,75,c16070,c16022,1,0,0,0,c16070


In [8]:
df['Fraud_chain']=0
df.loc[index_from_merge['index'],'Fraud']=1
df.loc[index_from_merge['index'],'Fraud_chain']=1
df['From->orig']=1
df['To->destination']=1

In [9]:
df.loc[index_from_merge['index'],:]

,orig,destination,A,B,C,Fraud,Fraud_chain,From->orig,To->destination
7,c16051,c16013,1,1,0,1,1,1,1
98,c16051,c16023,1,0,1,1,1,1,1
60,c16018,c16080,1,0,1,1,1,1,1
75,c16070,c16022,1,0,0,1,1,1,1


In [10]:
df

,orig,destination,A,B,C,Fraud,Fraud_chain,From->orig,To->destination
0,c16028,c16056,0,1,1,0,0,1,1
1,c16002,c16087,1,0,0,0,0,1,1
2,c16027,c16032,1,0,1,0,0,1,1
3,c16083,c16019,0,1,0,0,0,1,1
4,c16089,c16072,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
93,c16016,c16030,0,1,1,0,0,1,1
94,c16036,c16054,1,0,0,0,0,1,1
97,c16067,c16057,1,1,0,0,0,1,1
98,c16051,c16023,1,0,1,1,1,1,1


In [11]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [12]:
create_facts_and_examples(df_= df, target='Fraud', 
                          predicates=['A', 'B', 'C', 'From->orig', 'To->destination'], output_dir="examples/chain-fraud-erman")

Fraud
[ 7 10 28 35 41 44 46 56 60 75 76 90 98]
A
[ 1  2  4  5  6  7  8  9 10 13 19 21 22 25 26 27 28 30 32 34 35 37 38 41
 43 44 45 46 47 49 51 52 53 54 56 59 60 62 64 71 75 76 78 81 83 84 85 86
 87 88 90 91 94 97 98]
B
[ 0  3  7  8 10 16 18 20 21 22 23 24 26 27 28 29 31 32 35 40 41 44 46 47
 48 49 53 55 56 57 59 62 64 65 68 72 74 76 80 82 84 90 93 97]
C
[ 0  2  5  6 10 11 17 19 23 24 25 28 29 30 31 35 36 38 41 43 44 45 46 50
 51 52 54 55 56 57 60 61 65 68 69 71 72 76 78 80 81 85 88 89 90 91 93 98
 99]
From->orig
From->orig 0 16028
From->orig 1 16002
From->orig 2 16027
From->orig 3 16083
From->orig 4 16089
From->orig 5 16023
From->orig 6 16053
From->orig 7 16051
From->orig 8 16046
From->orig 9 16020
From->orig 10 16053
From->orig 11 16029
From->orig 13 16035
From->orig 16 16073
From->orig 17 16041
From->orig 18 16023
From->orig 19 16003
From->orig 20 16046
From->orig 21 16050
From->orig 22 16003
From->orig 23 16031
From->orig 24 16009
From->orig 25 16010
From->orig 26 16027
From->orig 